In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import glob     
import json
import os
from PIL import Image, ImageFilter
from skimage import exposure
import cv2


class Model:
    # 초기화
    def __init__(self, img_path, model_path, class_labels_path):
        self.img_path = img_path
        self.model_path = model_path
        self.class_labels_path = class_labels_path
        
        image_directory = img_path
        # jpg, png, jpeg 형식 (리스트형식으로 저장된다, 경로에는 파일한장만 존재하게)
        image_files = glob.glob(image_directory + '/*.png') + glob.glob(image_directory + '/*.jpg') + glob.glob(image_directory + '/*.jpeg')
        # 리스트중 첫번째것만 사용
        self.image_path = image_files[0]
        
        
    #예측 (전처리된이미지와 클래스라벨을 삽입)
    def preds_model(self, img, class_labels):
        model = tf.keras.models.load_model(self.model_path,  custom_objects={'KerasLayer': hub.KerasLayer})
        preds = model.predict(img) 
        
        # 결과값 상위 1개
        top_preds_idx = preds[0].argsort()[ : : -1][ : 1]
        top_preds_labels = [class_labels[idx] for idx in top_preds_idx]
        top_preds_probs = preds[0][top_preds_idx]
        return top_preds_labels, top_preds_probs
    
    #이미지전처리_1 (기본)
    def img_pre_default(self):  
        img = tf.keras.preprocessing.image.load_img(self.image_path, target_size=(256, 256))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)
        return img
    
    """
    #이미지전처리_2 (가우시안)
    def img_pre_gaussian(self): 
        img = Image.open(self.image_path)
        img = img.filter(ImageFilter.GaussianBlur(radius=4))
        img = img.resize((256, 256))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)
        return img
    
    #이미지전처리_3 (히스토그램 평활화)
    def img_pre_hist(self):
        img = tf.keras.preprocessing.image.load_img(self.image_path, target_size=(256, 256))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img.astype('float32') / 255.0
        img = exposure.equalize_hist(img)
        img = np.expand_dims(img, axis=0)
        return img
    
    #이미지전처리_4 (데이터증강)
    def img_pre_augmentation(self): 
        img = tf.keras.preprocessing.image.load_img(self.image_path, target_size=(256, 256))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img.astype('float32') / 255.0
        
        # 데이터 증강을 위한 ImageDataGenerator 생성
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1.0 / 255,
            rotation_range=40,
            width_shift_range=0.4,
            height_shift_range=0.4,
            shear_range=0.4,
            zoom_range=0.4,
            horizontal_flip=True,
            preprocessing_function=lambda img: tf.image.random_crop(img, [256, 256, 3])
        )
        
        # 데이터 증강 적용
        img = train_datagen.random_transform(img)
        
        img = np.expand_dims(img, axis=0)
        return img
    
    
    #이미지전처리_5 (이미지회전)
    def img_pre_rotate(self):
        img = tf.keras.preprocessing.image.load_img(self.image_path, target_size=(256, 256))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img.astype('float32') / 255.0

        # 회전 각도 설정
        angle = 75

        # 이미지 회전
        img = tf.keras.preprocessing.image.random_rotation(img, angle, row_axis=0, col_axis=1, channel_axis=2)
        img = np.expand_dims(img, axis=0)
        return img    
    
    
    #이미지전처리_6 (밝기조절)
    def img_pre_adjust_brightness(self):
        img = tf.keras.preprocessing.image.load_img(self.image_path, target_size=(256, 256))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img.astype('float32') / 255.0

        # 밝기 조절
        brightness_factor = 0.4
        img = tf.image.adjust_brightness(img, delta=brightness_factor)
        img = np.expand_dims(img, axis=0)
        return img
    
    #이미지전처리_7 (대비조절)
    def img_pre_adjust_contrast(self):
        img = tf.keras.preprocessing.image.load_img(self.image_path, target_size=(256, 256))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img.astype('float32') / 255.0

        # 대비 조절
        contrast_factor = 3.0
        img = tf.image.adjust_contrast(img, contrast_factor)
        img = np.expand_dims(img, axis=0)
        return img
    
    # 이미지전처리_8 (이미지 확대 및 축소)
    def img_pre_resize(self):
        img = tf.keras.preprocessing.image.load_img(self.image_path, target_size=(256, 256))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img.astype('float32') / 255.0

        # 이미지 크기 변경
        target_size = (128, 128)
        img = tf.image.resize(img, target_size)
        img = np.expand_dims(img, axis=0)
        return img
    """

    #라벨불러오기
    def class_label(self):
        class_labels = []
        with open(self.class_labels_path, 'r') as file:
            for line in file:
                class_labels.append(line.strip())
        return class_labels
    
    #결과출력
    def print_result(self, top_preds_labels, top_preds_probs):
        results = []
        #json형식 results
        for label, prob in zip(top_preds_labels, top_preds_probs):
            result = {
                '상품': label,
                '정확도': round(float(prob), 2)
            }
            results.append(result)
        print(results)
        return results
  
################################경로설정################################

# 이미지들어있는 폴더경로 
image_path = r'C:\Users\sj990\Desktop\ex4'

#분류기 모델경로, 분류기 라벨경로
clsf_model_path = r'C:\Users\sj990\Desktop\모델,라벨\모델\classification.h5'
clsf_label_path = r'C:\Users\sj990\Desktop\모델,라벨\라벨\classification_label.txt'

# 면류 모델경로, 면류 라벨경로
noodle_model_path = r'C:\Users\sj990\Desktop\모델,라벨\모델\model_noodle.h5'
noodle_label_path = r'C:\Users\sj990\Desktop\모델,라벨\라벨\model_noodle_label.txt'

# 미분류 모델경로, 면류 라벨경로
unclassified_model_path = r'C:\Users\sj990\Desktop\모델,라벨\모델\model_unclass.h5'
unclassified_label_path = r'C:\Users\sj990\Desktop\모델,라벨\라벨\model_unclass_label.txt'

################################경로설정################################

### 모델분류 ###
model_classification = Model(image_path, clsf_model_path, clsf_label_path)
class_labels = model_classification.class_label()
model_default_img = model_classification.img_pre_default()
classification_label, classification_prob = model_classification.preds_model(model_default_img, class_labels)


### 특징분류 ###
if(classification_label[0] == "라면"):
    #모델정의
    model_noodle = Model(image_path, noodle_model_path, noodle_label_path)
    #예측
    result_noodle_labels, result_noodle_probs = model_noodle.preds_model(model_noodle.img_pre_default(), model_noodle.class_label())
    #결과출력
    noodle_result = model_noodle.print_result(result_noodle_labels, result_noodle_probs)
    
    
    # json파일 저장필요할시 주석 제거후 사용
    with open('results_noodle.json', 'w', encoding='utf-8') as file:
        json.dump(noodle_result, file, ensure_ascii=False)
    
    
    '''
    # 사용한 이미지 파일 삭제 필요할시 주석 제거후 사용
    if results != None :
        os.remove(image_path)
    '''
    
elif(classification_label[0] =="미분류"):
    #모델정의
    model_unclassified = Model(image_path, unclassified_model_path, unclassified_label_path)
    #예측
    result_unclassified_labels, result_unclassified_probs = model_unclassified.preds_model(model_unclassified.img_pre_default(), model_unclassified.class_label())
    #결과출력
    unclassified_result = model_unclassified.print_result(result_unclassified_labels, result_unclassified_probs)
    
    
    # json파일 저장필요할시 주석 제거후 사용
    with open('results_unclassified.json', 'w', encoding='utf-8') as file:
        json.dump(unclassified_result, file, ensure_ascii=False)
    
    
    '''
    # 사용한 이미지 파일 삭제 필요할시 주석 제거후 사용
    if results != None :
        os.remove(image_path)
    '''